In [ ]:
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

import sys
sys.path.append("..")

from models.unet import GameUNet
from models.early_stopping import EarlyStopping

In [24]:
train = pd.read_pickle("../../dataset/train.pkl")
test = pd.read_pickle("../../dataset/test.pkl")
val = pd.read_pickle("../../dataset/val.pkl")

In [25]:
train = train.sample(1000)
test = test.sample(200)
val = val.sample(100)

In [30]:
def get_torch_tensor(x, y, a, num_classes: int, num_actions: int):
    x_tensor = torch.stack([
        tensor for tensor in x.map(lambda x: F.one_hot(x.to(torch.long), num_classes=num_classes))
    ]).permute(0, 3, 1, 2).to(torch.float)

    y_tensor = torch.stack([
        tensor for tensor in y.map(lambda x: F.one_hot(x.to(torch.long), num_classes=num_classes))
    ]).permute(0, 3, 1, 2).to(torch.float)

    a_tensor = torch.stack([
        tensor for tensor in a.map(lambda x: F.one_hot(torch.tensor(x), num_classes=num_actions))
    ]).to(torch.float)

    return x_tensor, y_tensor, a_tensor

In [33]:
x_train, y_train, a_train = get_torch_tensor(train['prev_state'], train['actual_state'], train['action'], num_classes=3, num_actions=4)
x_test, y_test, a_test = get_torch_tensor(test['prev_state'], test['actual_state'], test['action'], num_classes=3, num_actions=4)
x_val, y_val, a_val = get_torch_tensor(val['prev_state'], val['actual_state'], val['action'], num_classes=3, num_actions=4)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 32
train_loader = DataLoader(TensorDataset(x_train, a_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test, a_test, y_test), batch_size=batch_size, shuffle=False)
val_loader = DataLoader(TensorDataset(x_val, a_val, y_val), batch_size=batch_size, shuffle=False)

es = EarlyStopping(patience=10)

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GameUNet(n_classes=3, n_actions=4).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for x_batch, a_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        a_batch = a_batch.to(device)
        y_batch = y_batch.to(device)
        
        y_pred = model(x_batch, a_batch)
        loss = criterion(y_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x_batch.size(0)

    epoch_loss = running_loss / len(val_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}")

print("✅ Entrenamiento completado.")

a
a
a
a
Epoch [1/20] - Loss: 1.0392
a
a
a
a
Epoch [2/20] - Loss: 0.6792
a
a
a
a
Epoch [3/20] - Loss: 0.5033
a
a
a
a
Epoch [4/20] - Loss: 0.3985
a
a
a
a
Epoch [5/20] - Loss: 0.3223
a
a
a
a
Epoch [6/20] - Loss: 0.2712
a
a
a
a
Epoch [7/20] - Loss: 0.2267
a
a
a
a
Epoch [8/20] - Loss: 0.1964
a
a
a
a
Epoch [9/20] - Loss: 0.1667
a
a
a
a
Epoch [10/20] - Loss: 0.1443
a
a
a
a
Epoch [11/20] - Loss: 0.1274
a
a
a
a
Epoch [12/20] - Loss: 0.1132
a
a
a
a
Epoch [13/20] - Loss: 0.1024
a
a
a
a
Epoch [14/20] - Loss: 0.0918
a
a
a
a
Epoch [15/20] - Loss: 0.0831
a
a
a
a
Epoch [16/20] - Loss: 0.0747
a
a
a
a
Epoch [17/20] - Loss: 0.0698
a
a
a
a
Epoch [18/20] - Loss: 0.0637
a
a
a
a
Epoch [19/20] - Loss: 0.0594
a
a
a
a
Epoch [20/20] - Loss: 0.0559
✅ Entrenamiento completado.


In [68]:
model(torch.zeros((1, 3, 16, 16)), torch.zeros(1, 4)).shape

torch.Size([1, 3, 16, 16])

In [70]:
import torchsummary

torchsummary.summary(model, input_size=[(3, 16, 16), [4]])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]             864
       BatchNorm2d-2           [-1, 32, 16, 16]              64
              ReLU-3           [-1, 32, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]           9,216
       BatchNorm2d-5           [-1, 32, 16, 16]              64
              ReLU-6           [-1, 32, 16, 16]               0
        DoubleConv-7           [-1, 32, 16, 16]               0
            Linear-8                   [-1, 64]             320
              FiLM-9           [-1, 32, 16, 16]               0
        MaxPool2d-10             [-1, 32, 8, 8]               0
           Conv2d-11             [-1, 64, 8, 8]          18,432
      BatchNorm2d-12             [-1, 64, 8, 8]             128
             ReLU-13             [-1, 64, 8, 8]               0
           Conv2d-14             [-1, 6

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [61]:
%pip install torchsummary

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
1000 / 32